In [ ]:
# TODO: build data loader
# TODO: projection head
# TODO: implement loss
# TODO: train loop

In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import sys
project_path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl'
if project_path not in sys.path:
    sys.path.append(project_path)

In [8]:
# TODO: build data loader
## 1. load dataset
## 2. load model
##     - 2.1 get caption embeddings
##     - 2.2 get image embeddings
## 

# load dataset

import torch

# load dataset
path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/deliverables/datasets/f30k.pth'

with open(path, 'rb') as f:
    f30k_dataset = torch.load(f)

In [10]:
f30k_dataset[0]

('The man with pierce ears is wearing glasses and an orange hat .',
 <PIL.Image.Image image mode=RGB size=500x461>,
 125,
 25,
 '1007129816.jpg',
 0)

In [ ]:
# load model and embedd queries
from src.utils.utils import get_config, get_model, get_config_path, get_logger
from munch import Munch

def get_config(dataset, model):
    path = get_config_path(dataset, model)
    path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/config/align/f30k/development_local.yaml'
    with open(path, 'rb') as f:
        config = Munch.fromYAML(f)
        print('Loaded config path')
    return config

dataset = 'f30k'
model_name = 'align'
task = 't2i'
perturbation = 'random'
compute_from_scratch = False

config = get_config(model=model_name, dataset=dataset)

logging = get_logger(config=config)

config.logging = logging

model = get_model(config=config)

# update config
config.args = Munch(
    dataset=dataset,
    model=model, 
    task=task,
    perturbation=perturbation,
    compute_from_scratch=compute_from_scratch
    )


# get embeddings
caption_ids, captions, caption_emb = model.compute_caption_embeddings(ds_split=f30k_dataset)

In [106]:
model.compute_image_embeddings()

Loaded embeddings from: /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/precomputed/align/f30k-img-embeddings.pkl
Image embeddigns are already precomputed
Loaded precomputed filenames and embeddings from  /Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/precomputed/align/f30k-img-embeddings.pkl


(['2609797461.jpg',
  '1788892671.jpg',
  '129860826.jpg',
  '3376227992.jpg',
  '1423997242.jpg',
  '2083844357.jpg',
  '2830869109.jpg',
  '97138973.jpg',
  '725722798.jpg',
  '610841649.jpg',
  '1028205764.jpg',
  '4822402043.jpg',
  '4555563603.jpg',
  '4519904608.jpg',
  '3616638478.jpg',
  '271660510.jpg',
  '2641049916.jpg',
  '7393977570.jpg',
  '2319963351.jpg',
  '349198711.jpg',
  '2084622444.jpg',
  '7648640466.jpg',
  '3484070900.jpg',
  '1860543210.jpg',
  '8218123217.jpg',
  '3030793171.jpg',
  '3827171692.jpg',
  '1808504612.jpg',
  '4459723252.jpg',
  '4854547386.jpg',
  '3092370204.jpg',
  '104763189.jpg',
  '3701249979.jpg',
  '4537322302.jpg',
  '2905975229.jpg',
  '2768972186.jpg',
  '7294645258.jpg',
  '329456684.jpg',
  '4749087824.jpg',
  '3215896272.jpg',
  '3120921482.jpg',
  '2553550034.jpg',
  '157139628.jpg',
  '4775597702.jpg',
  '4752984291.jpg',
  '4509092468.jpg',
  '4834744984.jpg',
  '159845627.jpg',
  '4147681861.jpg',
  '3509224574.jpg',
  '47412966

In [14]:
# load precomputed embeddings
import pickle

path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-vl/datasets/f30k/precomputed/align/f30k-img-embeddings.pkl'

with open(path, 'rb') as f:
    data = pickle.load(f)


# prepare dictionary of image_filenames

from collections import defaultdict 

def def_value(): 
    return "Not Present"
      
img_filename_to_emb = defaultdict(def_value)

for filename, emb in zip(data[0], data[1]):
    # print(filename)
    # print(emb)
    # break
    img_filename_to_emb[filename] = emb


# prepare dictionary of caption embeddings
caption_id_to_emb = defaultdict(def_value)

for caption_id, emb in zip(caption_ids, caption_emb):
    caption_id_to_emb[caption_id] = emb

In [103]:
from torch.utils.data import DataLoader
from functools import partial

# generator for reproducibility
g = torch.Generator()
g.manual_seed(42)

def collate_fn(my_data, dicts):

    raw_capts, raw_imgs, caption_ids, image_ids, image_filenames, idx = zip(*my_data)

    img_emb_dict, caption_emb_dict = dicts

    # get image embeddings
    img_embs = [img_emb_dict[filename] for filename in image_filenames]
    img_embs = torch.stack(img_embs, 0)

    # get caption embeddings
    capt_embs = [caption_emb_dict[caption_id] for caption_id in caption_ids]
    capt_embs = torch.stack(capt_embs, 0)

    return raw_capts, raw_imgs, caption_ids, image_ids, image_filenames, idx, img_embs, capt_embs

f30k_dataloader = DataLoader(
    f30k_dataset,
    collate_fn=partial(
        collate_fn,
        dicts=(img_filename_to_emb, caption_id_to_emb)
        ),
    batch_size=8
)

batch = next(iter(f30k_dataloader))

In [128]:
len(f30k_dataset)

5000

In [104]:
raw_capts, raw_imgs, caption_ids, image_ids, image_filenames, idx, img_embs, capt_embs = batch

In [109]:
img_embs.shape

torch.Size([8, 640])

In [110]:
capt_embs.shape

torch.Size([8, 640])

In [114]:
p.forward(img_embs).shape

torch.Size([8, 2048])

In [115]:
capt_embs.shape[1]

640

In [130]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config.device = device

In [120]:
from src.models.projection_head import VLHead

multimodal_head = VLHead(
    image_input_dim=img_embs.shape[1], image_hidden_dim=2048, image_head_out_dim=2048,
    text_input_dim=capt_embs.shape[1], text_hidden_dim=2048, text_head_out_dim=2048,
    temperature=1
)

out = multimodal_head.forward(batch=batch)

In [126]:
# get optimizer

import itertools

params = [
    {
        'params': itertools.chain(
            model.image_projection_head.parameters(),
            model.text_projection_head.parameters()
        ),
        'lr': 1.0e-3,
        'weight_decay': 1.0e-3
    }
]
optimizer = torch.optim.AdamW(params)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    patience=1,
    factor=0.8
)

print('Created projection head, optimizer, and scheduler')

<generator object Module.parameters at 0x31fe9e7a0>

In [145]:
batch[6]

tensor([[-0.0029,  0.0269, -0.0061,  ..., -0.0318, -0.0864,  0.0348],
        [-0.0029,  0.0269, -0.0061,  ..., -0.0318, -0.0864,  0.0348],
        [-0.0029,  0.0269, -0.0061,  ..., -0.0318, -0.0864,  0.0348],
        ...,
        [ 0.0797, -0.0055, -0.0138,  ..., -0.0176, -0.0297, -0.0340],
        [ 0.0797, -0.0055, -0.0138,  ..., -0.0176, -0.0297, -0.0340],
        [ 0.0797, -0.0055, -0.0138,  ..., -0.0176, -0.0297, -0.0340]])

In [129]:
from src.train_valid import train_epoch, valid_epoch

epoch = 2

############################
# training and evaluation
############################
step = "epoch"
train_loss_values = []
valid_loss_values = []
best_loss = float('inf')
for epoch in range(epoch):
    running_loss = 0.0
    print(f"Epoch: {epoch + 1}")
    model.train()
    train_loss = train_epoch(config=config, model, train_loader, optimizer, lr_scheduler, step)
    # print(train_loss.avg)
    train_loss_values.append(train_loss.avg)
    
    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(config, model, f30k_dataloader)
        valid_loss_values.append(valid_loss.avg)

    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg

        # dt_string = get_dt_string()
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved Best Model to {best_model_path}! best loss - {best_loss}")
        log.info(f"Saved Best Model! best loss - {best_loss}")
        # torch.save(valid_loader, f"{CFG.deliverables_path}/{dt_string}_valid_loader.pth")
        # print('Saved the dataloader!')

    lr_scheduler.step(valid_loss.avg)
print('Finished training!')
log.info('Finished training!')

NameError: name 'config_train' is not defined